In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from keras.layers import LSTM, Dropout, Dense
from keras.layers import SimpleRNN
from keras.layers import GRU
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# imort Sequential
from keras.models import Sequential

**Read data**


In [19]:
import pandas as pd
df = pd.read_csv("/Users/rahuljauhari/Desktop/research-runoff/Data/merged_imd.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = df.iloc[:, :157]

df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)
monthly_mean = df.resample('M').mean()
print(monthly_mean.shape)

df_actual = pd.read_excel("/Users/rahuljauhari/Desktop/research-runoff/Data/Calibrated and Validated.xlsx")
# select last column
observed_runnoff = df_actual['observed']
# observed_runnoff.head()
print(observed_runnoff.shape)

(468, 156)
(468,)


In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def func(name,x,y):
    inv = 0

    if name == 'StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)
        y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler

    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0, 1))
        x_scaled = scaler.fit_transform(x)
        y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler
        
    return x, y, inv

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42))
]

In [22]:
activation = [ 'tanh']
optimizer = ['adam']
preprocess = ['StandardScaler']

In [23]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
for act in activation:
    for opt in optimizer:
        for pre in preprocess:
            x,y,inv_scaler= func(pre)
            X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,shuffle=False)

# ---------------------------
# Change from StackingClassifier to StackingRegressor
            meta_model = LinearRegression()
            model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
# ---------------------------

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred_train = model.predict(X_train)

            try:
                y = np.concatenate((y_pred_train, y_pred), axis=0)
                # inverse transform
                y_inv = inv_scaler.inverse_transform(y.reshape(-1, 1))
                # to csv
                pd.DataFrame(y_inv, columns=['Predicted']).to_csv('/Users/rahuljauhari/Desktop/research runoff/results1/imd_gru_lstm_0.2.csv', mode='a', header=True)
            except Exception as e:
                print(e)

/Users/rahuljauhari/opt/anaconda3/envs/tf1/lib/python3.9/site-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Cannot save file into a non-existent directory: '/Users/rahuljauhari/Desktop/research runoff/results1'


In [24]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

In [26]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_pred_train_inv = inv_scaler.inverse_transform(y_pred_train.reshape(-1, 1))
y_pred_test_inv = inv_scaler.inverse_transform(y_pred_test.reshape(-1, 1))
y_train__inv = observed_runnoff[:len(y_pred_train_inv)]
y_test__inv = observed_runnoff[len(y_pred_train_inv):]
print("KGE train: ", round(kge1(y_train__inv, y_pred_train_inv),4))
print("KGE test: ", round(kge1(y_test__inv, y_pred_test_inv),4))



KGE train:  0.8467
KGE test:  0.7176


In [27]:
y_train__inv = pd.DataFrame(y_train__inv)
y_pred_train_inv = pd.DataFrame(y_pred_train_inv)
y_train__inv.reset_index(drop=True, inplace=True)
y_pred_train_inv.reset_index(drop=True, inplace=True)
y_train__inv = pd.concat([y_train__inv,y_pred_train_inv],axis=1)
y_train__inv.to_csv(f'imd_stacking_train.csv',mode='a',header=True)
y_test__inv = pd.DataFrame(y_test__inv)
y_pred_test_inv = pd.DataFrame(y_pred_test_inv)
y_test__inv.reset_index(drop=True, inplace=True)
y_pred_test_inv.reset_index(drop=True, inplace=True)
y_test__inv = pd.concat([y_test__inv,y_pred_test_inv],axis=1)
y_test__inv.to_csv(f'imd_stacking_test.csv',mode='a',header=True)

In [31]:
df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_585.csv')
df_ssp['TimeDate'] = pd.to_datetime(df_ssp['TimeDate'])
df_ssp.set_index('TimeDate', inplace=True)
monthly_mean = df_ssp
print(monthly_mean.shape)
scaler = StandardScaler()
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)
# y=scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv = inv_scaler.inverse_transform(y_pred.reshape(-1, 1))
pd.DataFrame(y_inv).to_csv('SSP_monthly_585_imd_stacking.csv',header=True)

df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_370.csv')
df_ssp['TimeDate'] = pd.to_datetime(df_ssp['TimeDate'])
df_ssp.set_index('TimeDate', inplace=True)
monthly_mean = df_ssp
print(monthly_mean.shape)
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)
print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv = inv_scaler.inverse_transform(y_pred.reshape(-1, 1))
pd.DataFrame(y_inv).to_csv('SSP_monthly_370_imd_stacking.csv',header=True)

df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_245.csv')
df_ssp['DateTime'] = pd.to_datetime(df_ssp['DateTime'])
df_ssp.set_index('DateTime', inplace=True)
monthly_mean = df_ssp.resample('M').mean()
print(monthly_mean.shape)
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)
print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv1 = inv_scaler.inverse_transform(y_pred.reshape(-1, 1))
pd.DataFrame(y_inv1).to_csv('SSP_monthly_245_imd_stacking.csv',header=True)

df_ssp = pd.read_csv('/Users/rahuljauhari/Desktop/research-runoff/Data/SSP_monthly_126.csv')
df_ssp['DateTime'] = pd.to_datetime(df_ssp['DateTime'])
df_ssp.set_index('DateTime', inplace=True)
monthly_mean = df_ssp
print(monthly_mean.shape)
x,y,inv_scaler= func('StandardScaler',monthly_mean,observed_runnoff)
print(x.shape)
y_pred = model.predict(x)
print(y_pred.shape)
y_inv = inv_scaler.inverse_transform(y_pred.reshape(-1, 1))
pd.DataFrame(y_inv).to_csv('SSP_monthly_126_imd_stacking.csv',header=True)


(948, 156)
(948, 156)
(948,)
(948, 156)
(948, 156)
(948,)
(948, 156)
(948, 156)
(948,)
(948, 156)
(948, 156)
(948,)
